In [1]:
import csv
import json
from wikiparse import indexer, syntax_parser as sp
import time

In [13]:
dumps = indexer.load_dumps(build_index=False, scratch_folder='py3')
english = dumps['en']

opening F:/enwiki-20190101-pages-articles-multistream.xml/scratch\py3\index.db
current mapping 19.1 m pages

__init__ complete


In [14]:
c = english.cursor

Let's get an estimate of how long it's going to take to extract coordinates for every article.

In [4]:
entries = []
sample_factor = 1 /10000
ts = time.time()
for i in range(int(english.metadata['size'] * sample_factor)):
    page = english.get_page_by_index(i+1)
    tags = sp.get_tags(page.text, 'Coord')
    title = page.title
    if tags:
        for tag in tags:
            entries.append((title,tag))
#             print(title, tag)
total_time = time.time() - ts
print(round(total_time, 2), 'seconds for', int(english.metadata['size'] * sample_factor), 'samples taken')
print('estimated total time:', round(total_time / sample_factor / 60 / 60, 2), 'hours')

1.53 seconds for 1909 samples taken
estimated total time: 4.26 hours


In [5]:
len(entries)

80

Percentage of articles with coordinates associated with them:

In [6]:
round(100 * len(entries) / int(english.metadata['size'] * sample_factor), 2)

4.19

In [12]:
# start = int(19096287 * 0.01)
start = 19090000
sample_factor = 1
ts = time.time()
print(start, '-', int(english.metadata['size'] * sample_factor))
for i in range(start, int(english.metadata['size'] * sample_factor)):
    try:
        page = english.get_page_by_index(i)
    except:
        break
    if page.text is None:
        continue
    tags = sp.get_tags(page.text, 'Coord')
    title = page.title
    if tags:
        tagstring = '||'.join(tags)
        c.execute(
            f'UPDATE indices SET coords=? WHERE page_num=?',
            (tagstring, i+1)
        )
    if i % 100 == 0:
        english.db.commit()
    if i % 1000 == 0:
        print(round(100*i/(english.metadata['size'] * sample_factor), 2), '%', f'({i})', end='\r')
total_time = time.time() - ts
print(round(total_time, 2), 'seconds for', int(english.metadata['size'] * sample_factor), 'samples taken')
print('estimated total time:', round(total_time / sample_factor / 60 / 60, 2), 'hours')

19090000 - 19096287
0.37 seconds for 19096287 samples taken
estimated total time: 0.0 hours


Sample some of the coords:

In [15]:
c.execute('''
    SELECT title,coords FROM indices WHERE page_num > 19000000 AND coords != ""
''').fetchall()

[('Egyptian constitutional referendum, 2011',
  'Coord|30|36|05|N|84|39|41|W|type:waterbody_region:US-FL|display=inline||Coord|30|26|49|N|84|33|22|W|type:waterbody_region:US-FL|display=inline,title'),
 ('Wikipedia:Miscellany for deletion/Draft:Alana Ribble',
  'Coord|...||Coord|19.9718|79.9296|display=inline,title'),
 ('Kuma von Clifford', 'Coord|LAT|LONG|type:landmark|display=inline'),
 ('Wikipedia:WikiProject Spam/LinkReports/kore25.com',
  'Coord|51|08|56|N|001|34|12|W|region:GB_type:airport|display=inline,title'),
 ('1993 Bank of the West Classic – Doubles',
  'Coord|5|21|31.9|N|100|18|20.9|E|display=title'),
 ('Timeline of the presidency of Bill Clinton (1996)',
  'Coord|LAT|LONG|type:landmark|display=inline'),
 ('Portal:Pan-Africanism/Selected history/3',
  'Coord|29.9746185|N|90.0746852|W|type:landmark|display=title'),
 ('Draft:Sundial Aeon', 'Coord|LAT|LONG|display=inline,title'),
 ('Category:Legal comedy films',
  'Coord|LAT|LONG|type:landmark|display=inline'),
 ('Die blauen S

Count all of the entries with coords

In [17]:
coordsCount = c.execute('''
    SELECT Count(*) FROM indices WHERE coords != ""
''').fetchone()[0]

In [18]:
coordsCount

136447

Compute percentage of articles that have associated coordinates

In [19]:
round(100 * coordsCount / 19096287, 2)

0.71